In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
len(docs_raw)

3

In [5]:
# there are 3 courses in the raw file
# we can check the length of the documents in each course
# in each document, it has 3 components: 'text', 'section','question'
documents = []

for course_dic in docs_raw:
    print(course_dic['course'])
    print(len(course_dic['documents']))
    for doc in course_dic['documents']:
        # add one more component in each document: 'course"
        doc['course'] = course_dic['course']
        documents.append(doc)

data-engineering-zoomcamp
435
machine-learning-zoomcamp
375
mlops-zoomcamp
138


In [25]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
q = 'the course has already started, can I still enroll?'

In [26]:
index.fit(documents)

In [27]:
# boost when we think "one parameter contains the keywords" is more important than the other one
# the question term is 3 times more important than the text term 
# the function of index.search is to only return relative context in the documents collection

boost = {'question': 3.0, 'section': 0.5}

index.search(
    query = q,
    boost_dict = boost,
    num_results = 5
)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [28]:
results = index.search(
    query = q,
    filter_dict = {'course': 'data-engineering-zoomcamp'}, # only focus on 'data-engineering-zoomcamp'
    boost_dict = boost,
    num_results = 5
)

In [36]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [29]:
from openai import OpenAI

In [48]:
import os
os.environ

In [31]:
# there should be a api_key = "XXXXXX" in (), for security reason, you need to delect it to git push
# refer to tutorial video 1.2, 1.3
# client = OpenAI()

In [49]:
response = client.chat.completions.create(model = 'gpt-4o',
                                          messages = [{'role': 'user',
                                                       'content': q}]
                                             )
response

In [50]:
response.choices[0].message.content

In [34]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION base on the CONTEXT. 
Only use the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [35]:
results[0]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [37]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer:{doc['text']}\n\n"

print(context)

section: General course-related questions
 question: Course - Can I still join the course after the start date?
 answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
 question: Course - Can I follow the course after it finishes?
 answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
 question: Course - When will the course start?
 answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start 

In [38]:
prompt = prompt_template.format(question = q, context = context).strip()
print(prompt)

You're a course teaching assistant. Answer the QUESTION base on the CONTEXT. 
Only use the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
 question: Course - Can I still join the course after the start date?
 answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
 question: Course - Can I follow the course after it finishes?
 answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: Gene

In [39]:
response = client.chat.completions.create(model = 'gpt-4o',
                                          messages = [{'role': 'user',
                                                       'content': prompt}]
                                             )
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

### Clean the data flow and modulatizion

In [43]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'}, # only focus on 'data-engineering-zoomcamp'
        boost_dict = boost,
        num_results = 5
    )

    return results

In [40]:
def build_prompt(query, searching_result):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION base on the CONTEXT. 
    Only use the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in searching_result:
        context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer:{doc['text']}\n\n"

    prompt = prompt_template.format(question = query, context = context).strip()

    return prompt

In [41]:
def llm(prompt):
    response = client.chat.completions.create(model = 'gpt-4o',
                                          messages = [{'role': 'user',
                                                       'content': prompt}]
                                             )
    return response.choices[0].message.content

In [44]:
query = "how do I run Kafka"      
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

answer

'To run Kafka, specifically a Java Kafka producer or consumer, you can execute the following command in the terminal from the project directory:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the appropriate name of your jar file.'

In [45]:
query = "the course has already started, can I still enroll?"      
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

answer

"Yes, you can still enroll in the course after it has started. Even if you don't register, you are still eligible to submit the homework assignments. However, be mindful of the deadlines for turning in the final projects and avoid leaving everything to the last minute."

In [46]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer
    
query = "how do I run Kafka"          
rag(query)

'To run Kafka, based on the provided CONTEXT, you can execute the following command in the terminal within the project directory to start a Java Kafka producer:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file. This command will run a Java-based Kafka producer as part of Module 6 in your course.'

In [51]:
from elasticsearch import Elasticsearch